RNN
Исследование применения рекуррентных сетей для рекомендаций.
Используя инструментарий https://github.com/rdevooght/sequence-based-recommendations сравнить представленные модели 
- Recurrent Neural Networks 
Layers: (подобрать ячейку (LSTM, GRU, Vanilla), количество и мощность слоев, проверить эффективность использования embeddings и двунаправленности )
Update mechanism (оптимизаторы, learning rate)
Noise (Dropout)
Other options (размер миги-батчей, последовательностей)
(! Строим рекомендации с учетом категорий в которых пользователь уже покупал - необходимо обучать модели с параметром -- repeated_interactions )
- Latent Trajectory Modeling
- Bayesian personalized ranking (BPR-MF)
- Fusing Similarity Models with Markov Chains for Sparse Sequential Recommendation (Fossil)

Метрики:
recall, sps, item_coverage, user_coverage

В качестве простых baselines использовать Markov Chain, KNN, POP



Оценить применимость и оптимальность этих моделей для решения задачи рекомендаций отдельно в случае short term prediction (sps) и long term prediction (recall).


В соответствии с комментариями преподавателя можно рассмотреть только RNN

In [1]:
import pandas as pd

In [33]:
df = pd.read_csv('TransactionsOTUS.csv', sep=',', names=list('ritu'), index_col=False,usecols=range(4), engine='python')

In [34]:
df.head()

,r,i,t,u
0,2.0,4814,2018-09-28 23:11:22.0000000,2819090
1,2.0,4814,2018-09-28 20:50:12.0000000,3803173
2,2.0,4814,2018-09-28 20:54:09.0000000,6087397
3,2.0,4814,2018-09-28 20:58:12.0000000,6087397
4,2.0,4814,2018-09-28 21:25:12.0000000,5409942


In [11]:
! python "C:\Users\1\projects\ipynb\otus\32 (rnn 2)\homework\sequence-based-recommendations-master\preprocess.py" --help

usage: preprocess.py [-h] -f FILENAME [--columns COLUMNS] [--sep SEP]
                     [--min_user_activity MIN_USER_ACTIVITY]
                     [--min_item_pop MIN_ITEM_POP] [--val_size VAL_SIZE]
                     [--test_size TEST_SIZE] [--seed SEED]

optional arguments:
  -h, --help            show this help message and exit
  -f FILENAME           Input file
  --columns COLUMNS     Order of the columns in the file (eg: "uirt"), u for
                        user, i for item, t for timestamp, r for rating. If r
                        is not present a default rating of 1 is given to all
                        interaction. If t is not present interactions are
                        assumed to be in chronological order. Extra columns
                        are ignored. Default: uit
  --sep SEP             Separator between the column. If unspecified pandas
                        will try to guess the separator
  --min_user_activity MIN_USER_ACTIVITY
                     

In [51]:
! python "C:\Users\1\projects\ipynb\otus\32 (rnn 2)\homework\sequence-based-recommendations-master\preprocess.py" -f "C:\Users\1\projects\ipynb\otus\32 (rnn 2)\homework\TransactionsOTUS.csv" --columns ritu --sep ',' 

This program will create a lot of files and directories in C:\Users\1\projects\ipynb\otus\32 (rnn 2)\homework/
Load data...
Sort data in chronological order...
Remove inactive users and rare items...
            r     i          t        u
3498175  0.52  5964 2018-09-01  6076456
Map original users and items ids to consecutive numerical ids...
Save ids mapping to file...
Split data into training, validation and test sets...
Save training, validation and test sets in the triplets format...
Save the training set in the sequences format...
Save the validation set in the sequences format...
Save the test set in the sequences format...
Save the extended training set in the sequences format...
Save stats...
Data ready!
            r    i          t       u u_original i_original
3498175  0.52  148 2018-09-01  399066    6076456       5964
3476257  3.90  148 2018-09-01  298765    5254295       5964
3476258  5.05  148 2018-09-01  287408    5183101       5964
3476259  0.80  148 2018-09-01  123258 

Выполнили подготовку данных

In [82]:
! python ".\sequence-based-recommendations-master\train.py" -d "./" -m RNN

Compiling train...
Compilation done.
Compiling test...
Compilation done.
Opening file (1)


C:\ProgramData\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Traceback (most recent call last):
  File ".\sequence-based-recommendations-master\train.py", line 60, in <module>
    main()
  File ".\sequence-based-recommendations-master\train.py", line 57, in main
    validation_metrics=args.metrics.split(','))
  File "C:\Users\1\projects\ipynb\otus\32 (rnn 2)\homework\sequence-based-recommendations-master\neural_networks\rnn_base.py", line 289, in train
    batch = next(batch_generator)
  File "C:\Users\1\projects\ipynb\otus\32 (rnn 2)\homework\sequence-

Дальнейшая работа выполнялась в docker контейнере, так как слишком много ошибок. Результаты по валидации. Тестовые результаты не работают

UserKNN, MM и другие простые модели НЕ реализованы в данной версии и поэтому рассматриваются только RNN

Результат выполнения:

Теперь тоже самое выполним для repeated

python train.py -d ../data/ --max_iter 20 --repeated_interactions -m RNN

Гораздо лучше)

Обучим аналогично LSTM

Лучше. Используем как основную дальше, но добавим двунаправленность

Выигрыш есть, но очень небольшой. Теперь посмотрим embedding

Самое лучшее значение. Добавим слоев

не помогло, скорее всего переобучается

Изменим на adagard

In [ ]:
python train.py -d ../data/ --max_iter 20 --repeated_interactions --r_emb 50 --r_l 32-32 --r_t LSTM -m RNN

Добавим dropout

немного улучшилось

Изменим размер батча

ничего не изменилось. Теперь попробуем BRM-MF

In [ ]:
python train.py -d ../data/ --max_iter 20 --repeated_interactions -m BPR-MF

Плохо. Теперь попробуем Fossil

Похоже надо настраивать. Наибольшее качество получено на  python train.py -d ../data/ --max_iter 20 --repeated_interactions --n_dropout 0.2 --r_emb 50 --r_t LSTM -m RNN